In [125]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import random
from statsmodels.stats.power import tt_ind_solve_power

def date_converter(x):
    return datetime.datetime.strptime(x,'%Y-%m-%d')

In [126]:
df = pd.read_csv('user_amount_var_reduction.csv')
df['event_date']=df.event_date.apply(date_converter)

## answer

In [127]:
#изначальный
df

,event_date,user_id,user_segment,category,region,revenue_amount
0,2019-06-18,4129257975403696,4626944681007198896,7204754148424990691,2201388710798473628,139
1,2019-06-18,7225254357470025,4626944681007198896,57514010642945023,3048517866726460689,193
2,2019-06-18,8912846500600569,4626944681007198896,5983323600580891431,3048517866726460689,160
3,2019-06-18,20910801279823046,4626944681007198896,6847326422676498281,7762196236856042106,209
4,2019-06-18,34873300821657220,2376074197230840906,3187769798308634693,3855810523611026650,160
...,...,...,...,...,...,...
8625837,2019-12-08,9170823997728769058,2376074197230840906,4658595883120885993,3855810523611026650,133
8625838,2019-12-08,9178869387857307556,4626944681007198896,1472231361867825698,3048517866726460689,63
8625839,2019-12-08,9186690404771258365,2376074197230840906,4546178272531746227,8867014108346120905,168
8625840,2019-12-08,9195678548159495844,2376074197230840906,5983323600580891431,1187552825302355021,118


In [128]:
dataset = df.head(100000)

In [165]:
def choose(data, priznak, start_date, delta, share, share_bool):
    """
    в функцию подается датафрейм
    она фильтрует по датам
    группирует пользователей по user_id, в местах где одному пользовтаелю соответствует несколько групп 
    выбирается самая популярная группа, при равенстве выбирается группа у которой средняя цена покупки выше
    """
    data = data[['event_date', 'user_id', priznak, 'revenue_amount']]
    #берём нужные даты
    df_strata = data[(data.event_date>=start_date)&(data.event_date<=start_date+datetime.timedelta(days=delta))]

    #part1
    #отрезаем тез пользователей у которых только одна страта

    #список где одна категория на одного юзера -> делаем inner join c таблицей osn
    dub0 = (df_strata.groupby(['user_id', priznak]).count().reset_index().groupby(['user_id']).count()
     .revenue_amount == 1).reset_index()[(df_strata.groupby(['user_id', priznak]).count().reset_index().groupby(['user_id']).count().revenue_amount == 1).reset_index().revenue_amount == True][['user_id']]

    dd_part1 = df_strata.merge(dub0, left_on = 'user_id', right_on = 'user_id', how = 'inner')

    osn = df_strata.groupby(['user_id']).sum().reset_index()[['user_id', 'revenue_amount']]

    dub0_rev_part1 = dub0.merge(osn, left_on = 'user_id', right_on = 'user_id', how = 'inner')

    part1 = dd_part1.merge(dub0_rev_part1, left_on = 'user_id', right_on = 'user_id', how = 'inner')[['user_id', priznak, 'revenue_amount_y']]
    part1.rename(columns = {'revenue_amount_y': 'revenue_amount'}, inplace = True)

    part1 = part1.drop_duplicates()
    
    
    #part2
    #отрезаем тех пользователей у которых больше одной страты
    
    peop = df_strata.groupby(['user_id', priznak]).count()
    peop.reset_index(inplace = True)

    dub = (df_strata.groupby(['user_id', priznak]).count().reset_index().groupby(['user_id']).count()
     .revenue_amount != 1).reset_index()[(df_strata.groupby(['user_id', priznak]).count().reset_index().groupby(['user_id']).count().revenue_amount != 1).reset_index().revenue_amount == True]

    part2_rev = dub.merge(df_strata, left_on = 'user_id', right_on = 'user_id', how= 'inner').groupby('user_id')[['revenue_amount_y']].sum().reset_index()
    part2_rev.rename(columns = {'revenue_amount_y': 'revenue_amount'}, inplace = True)

    ob = peop.merge(dub, left_on = 'user_id', right_on = 'user_id')

    max_group = ob.groupby(['user_id']).event_date.max().reset_index()

    new = ob.merge(max_group, left_on = 'user_id', right_on = 'user_id', how = 'inner')
    new = new[['user_id', priznak, 'event_date_x', 'event_date_y']]
    new.rename(columns = {'event_date_x': 'count', 'event_date_y': 'count_max'}, inplace = True)

    new = new[new['count'] == new['count_max']] # убираем те что меньше макс

    #отсортированное соответствие между index и priznak по средней revenue_amount
    segment_mean = df_strata.groupby(priznak).revenue_amount.mean().sort_values(ascending = True).reset_index().reset_index()

    index_user = new.merge(segment_mean, left_on = priznak, right_on = priznak). \
    groupby('user_id').index.max().reset_index()

    part2 = part2_rev.merge(index_user, left_on = 'user_id', right_on = 'user_id', how = 'inner'). \
    merge(segment_mean[['index', priznak]], left_on = 'index', right_on = 'index', how = 'inner')[['user_id',priznak, 'revenue_amount']]
    
    # объедниняем и перемешиваем 
    part_12 = pd.concat([part1, part2])
    part_12 = part_12.sample(part_12.shape[0], random_state = 7575)
    
    #Нам нужно разбиение, которое сохраняем пропорции страт в контрольной и тестовой группах
    part_12['group'] = part_12.groupby(priznak).cumcount()
    part_12 = part_12.merge(part_12.groupby(priznak).group.max().reset_index(), left_on = priznak, right_on = priznak, how = 'inner')
    
    if share_bool:
        part_12 = part_12[round(part_12['group_y']*share) >= part_12['group_x']]
    
    return part_12

In [163]:
data = choose(data = df, priznak = 'region',start_date = datetime.datetime(2019, 8, 1, 0, 0), delta = 30, share = 0.1, share_bool = True)

In [164]:
data

,user_id,region,revenue_amount,group_x,group_y
0,6427656744383745604,8867014108346120905,615.0,0,18637
1,5253112167057242451,8867014108346120905,258.0,1,18637
2,4064231596859176816,8867014108346120905,69.0,2,18637
3,4748994501792486977,8867014108346120905,342.0,3,18637
4,350020614556563818,8867014108346120905,782.0,4,18637
...,...,...,...,...,...
599889,6228997776374976707,8666105090637809898,832.0,3,50
599890,6901591664329069611,8666105090637809898,21.0,4,50
599891,66830500203754048,8666105090637809898,64.0,5,50
599937,8566507811977528407,928013360687874809,109.0,0,14


In [130]:
def std_strat(data, priznak):
    strata_p = data.groupby([priznak]).count()/data.count()[0]
    data_test=data.join(strata_p, on=priznak, rsuffix='_probality')
    
    std = float(((data_test.groupby([priznak]).var().revenue_amount*data_test.groupby([priznak]).mean().revenue_amount_probality).sum()) ** 0.5)
    return std

In [131]:
from statsmodels.stats.power import tt_ind_solve_power

In [167]:
priznaki = ['user_segment', 'category', 'region']
for priz in priznaki:
    print('---------------------------')
    print('Strata:',  priz)
    data = choose(data = df, priznak = priz, start_date = datetime.datetime(2019, 8, 1, 0, 0), 
                  delta = 30, share = 0.5, share_bool = True) #получаем необходимую подвыборку

    mean = data.revenue_amount.mean()
    std = data.revenue_amount.std()

    strata_p = data.groupby([priz]).count()/data.count()[0]
    data_test=data.join(strata_p, on=priz, rsuffix='_probality')

    std_strat = ((data_test.groupby([priz]).var().revenue_amount*data_test.groupby([priz]).mean().revenue_amount_probality).sum()) ** 0.5
    mean_strat =(data_test.groupby([priz]).mean().revenue_amount*data_test.groupby([priz]).mean().revenue_amount_probality).sum()
    
    alpha = 0.01
    power = 0.8
    result_effect = 0.05

    effect_size = (mean /std) * result_effect
    effect_size_strat = (mean_strat /std_strat) * result_effect
    
    sample_vol = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, nobs1=None, ratio=1, alternative = 'two-sided')
    sample_vol_strat = tt_ind_solve_power(effect_size=effect_size_strat, alpha=alpha, power=power, nobs1=None, ratio=1, alternative = 'two-sided')

    print('Usually:', sample_vol/data.shape[0])
    print('Strat:', sample_vol_strat/data.shape[0])



---------------------------
Strata: user_segment
Usually: 1.5133413842040477
Strat: 1.3969360072163266
---------------------------
Strata: category
Usually: 1.534230074916239
Strat: 1.5196654547174167
---------------------------
Strata: region
Usually: 1.2553140850917903
Strat: 1.2499143201277616


Существенных снижений не было зафиксировано, но лучше всех себя показало разбиение по user_segment(дало максимальное уменьшение относительно обычного случая с простым подсчетом дисперсии)